In [11]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [12]:
tidy_character <- read_csv("data/datasource.csv", skip=1, col_names = c("label",seq(from=1,to=784,by=1)))|>
                    mutate(label=as.factor(label))
tidy_character 

Rows: 11176 Columns: 785
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (785): label, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


label,1,2,3,4,5,6,7,8,9,⋯,775,776,777,778,779,780,781,782,783,784
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
25,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [13]:
set.seed(123)
initialsplit <- tidy_character|>
                    initial_split(prop=0.1,strata=label)

initialsplit<- training(initialsplit)
character_split <-  initialsplit|>
                    initial_split(prop=0.75,strata=label)

traning_character <- training(character_split)
testing_character <- testing(character_split)
training_rows<-nrow(traning_character)
testing_rows<-nrow(testing_character)


In [14]:
traning_character<-tibble(`#`=seq(from=1,to=training_rows,by=1))|>
                   bind_cols(traning_character)
testing_character<-tibble(`#`=seq(from=1,to=testing_rows,by=1))|>
                   bind_cols(testing_character)
traning_character
testing_character

#,label,1,2,3,4,5,6,7,8,⋯,775,776,777,778,779,780,781,782,783,784
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
835,24,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
836,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
837,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


#,label,1,2,3,4,5,6,7,8,⋯,775,776,777,778,779,780,781,782,783,784
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
278,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
279,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
280,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [43]:
set.seed(123)
vfold<-traning_character|>
        vfold_cv(v=5,strata=label)

knn_spec<-nearest_neighbor(weight_func="rectangular",neighbors=tune())|>
            set_engine("kknn")|>
            set_mode("classification")

chr_recipe<-recipe(label~., data = traning_character)
            

k_vals<-tibble(neighbors=seq(from=1,to=100,by=5))



In [ ]:
fit<-workflow()|>
     add_recipe(chr_recipe)|>
     add_model(knn_spec)|>
     tune_grid(resamples=vfold,grid=k_vals)|>
     collect_metrics()|>
     filter(.metric=="accuracy")
fit

In [18]:
traning_character
n_model<-glm(label ~., data = traning_character, family = binomial)

#,label,1,2,3,4,5,6,7,8,⋯,775,776,777,778,779,780,781,782,783,784
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
835,24,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
836,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
837,25,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [21]:
summary<-summary(n_model)


In [33]:
pred_test <- predict(n_model,testing_character,type="response")
pred_test

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


1            2            3            4            5            6 
8.985768e-01 9.999999e-01 1.000000e+00 9.999916e-01 4.674581e-06 1.481934e-05 
           7            8            9           10           11           12 
1.000000e+00 9.902170e-02 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          13           14           15           16           17           18 
1.000000e+00 1.000000e+00 2.220446e-16 9.883116e-01 1.000000e+00 7.280585e-11 
          19           20           21           22           23           24 
1.000000e+00 9.999889e-01 1.000000e+00 1.000000e+00 1.000000e+00 1.826561e-02 
          25           26           27           28           29           30 
1.000000e+00 9.954611e-01 1.000000e+00 3.065788e-07 1.000000e+00 1.000000e+00 
          31           32           33           34           35           36 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 9.999971e-01 1.000000e+00 
          37           38           39           40           41           42 
1.000000e+00 1.000000e+00 1.193784e-03 1.000000e+00 1.000000e+00 9.996318e-01 
          43           44           45           46           47           48 
5.138283e-05 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          49           50           51           52           53           54 
1.000000e+00 1.000000e+00 2.220446e-16 1.000000e+00 1.000000e+00 1.000000e+00 
          55           56           57           58           59           60 
1.000000e+00 1.000000e+00 1.000000e+00 9.847293e-01 1.000000e+00 1.000000e+00 
          61           62           63           64           65           66 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          67           68           69           70           71           72 
1.000000e+00 1.840257e-03 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          73           74           75           76           77           78 
1.000000e+00 1.074071e-01 3.085646e-05 1.000000e+00 1.000000e+00 1.000000e+00 
          79           80           81           82           83           84 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          85           86           87           88           89           90 
1.000000e+00 9.999998e-01 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
          91           92           93           94           95           96 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 9.982629e-01 1.000000e+00 
          97           98           99          100          101          102 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
         103          104          105          106          107          108 
7.112784e-08 1.000000e+00 1.000000e+00 1.000000e+00 9.407267e-01 1.000000e+00 
         109          110          111          112          113          114 
1.000000e+00 9.999999e-01 1.000000e+00 4.822612e-01 1.000000e+00 7.012697e-01 
         115          116          117          118          119          120 
1.287704e-01 9.902039e-01 1.000000e+00 9.058030e-01 9.999999e-01 1.000000e+00 
         121          122          123          124          125          126 
1.000000e+00 9.999999e-01 1.000000e+00 1.000000e+00 9.999872e-01 1.000000e+00 
         127          128          129          130          131          132 
1.000000e+00 1.000000e+00 3.037704e-11 3.908547e-02 1.000000e+00 1.000000e+00 
         133          134          135          136          137          138 
1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 
         139          140          141          142          143          144 
1.000000e+00 9.999919e-01 9.998432e-01 9.999999e-01 1.357745e-01 1.000000e+00 
         145          146          147          148          149          150 
9.999791e-01 1.000000e+00 4.778130e-03 1.000000e+00 6.845907e-01 1.000000e+00 
         151          152          153          154          

In [37]:
pred.class<-ifelse(pred_test>=0.7, 1, 0)

In [38]:
table(pred.class)

pred.class
  0   1 
 35 245 

In [42]:
test_with_prob<-data.frame(probability=pred_test)|>
                bind_cols(testing_character)
test_with_prob

,probability,#,label,1,2,3,4,5,6,7,⋯,775,776,777,778,779,780,781,782,783,784
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.8985768,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0.9999999,2,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,1.0000000,3,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
278,1,278,25,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
279,1,279,25,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
280,1,280,25,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
